# Project week 3

In [ ]:
#Libraries
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import random
import time
import ipywidgets as widgets
from IPython.display import Javascript, display
from difflib import SequenceMatcher
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

#credencials

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
        client_id='7b1cccc6a20546aa819d65942b6e98e3',
        client_secret='47a9a88346464b52823ade52b4be19af'))

In [ ]:
#functions

def run_all(ev):
    display(Javascript("""IPython.notebook.execute_cell_range(
                       IPython.notebook.get_selected_index()+1,
                       IPython.notebook.get_selected_index()+2)"""))

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

#widgets
yes_or_no = widgets.Dropdown(
            options = ['','Yes','No'],   
            disabled = False,
            )

button = widgets.Button(button_style='info',description="Submit")
button.on_click(run_all)

In [ ]:
#hot charts
hot_charts = 'https://www.billboard.com/charts/hot-100'
response = requests.get(hot_charts)
charts = BeautifulSoup(response.content, 'html.parser')


In [ ]:
#DataFrame Billaboard

songs = []

for music in charts.select(".chart-element__information__song"):
    songs.append(music.get_text())
    
artists = []

for artist in charts.select(".chart-element__information__artist"):
    artists.append(artist.get_text())
    
rankings = []

for rank in charts.select(".chart-element__rank__number"):
    rankings.append(rank.get_text())
    
other_rankings = []

for other in charts.select(".chart-element__meta"):
    other_rankings.append(other.get_text())
    
last_week = other_rankings[0::3]
peak = other_rankings[1::3]
wks_on_chart = other_rankings[2::3]

top_charts = pd.DataFrame({
    "Song": songs,
    "Artist":artists,
    "Position":rankings,
    "Last Week Position":last_week,
    "Peak": peak,  
    "Weeks on Chart": wks_on_chart
})


In [ ]:
#Getting lower case
songs_from_billboard = []

for song in top_charts['Song']:
    songs_from_billboard.append(song.lower())

In [ ]:
#Big DataFrame & Clusters
features = pd.read_csv('dataprocess/features_df.csv')
all_features = features.copy()
features.drop(columns=['Unnamed: 0','id','uri','analysis_url'],inplace=True)
features.drop(columns=['type','track_href'], inplace=True)
scaler = StandardScaler()
x_scaled = scaler.fit_transform(features)
x_scaled_df = pd.DataFrame(x_scaled)
kmeans = KMeans(n_clusters=8, random_state=1234)
kmeans.fit(x_scaled)
clusters = kmeans.predict(x_scaled)
all_features["cluster"] = clusters

In [ ]:
#Selecting a Song

selection = input('Select a song:').lower()

selection

random_1 = random.randint(1,100)
random_2 = random.randint(1,100)
random_3 = random.randint(1,100)
random_4 = random.randint(1,100)

for music in songs_from_billboard:
    if similar(music, selection)== 1:
        break
    elif similar(music, selection)>= 0.80:
        selection=music
        print('Your Chosen song was:', selection)

if selection in songs_from_billboard:
    print('We would like to recomend you this song: ', top_charts['Song'].loc[random_1], 'by', top_charts['Artist'].loc[random_1])
    time.sleep(2)
    print('Would you like to have another suggestion?')
    display(yes_or_no)
    display(button)
else:  
    print('We cannot find a Hot Chart song that might match with your taste, but we can provide some suggestions according to the song features you have chosen')
    print('Do you want to continue?')
    display(yes_or_no)
    display(button)
    
    

In [ ]:
list_feedback = []

if yes_or_no.value == 'No':
    print("Thanks. See you next time.")
else:
    print('We would like to suggest you the following songs according to the chosen song features:')
    results = spotify.search(q=selection, limit = 10)
    features_user = results['tracks']['items'][0]['uri']
    feats = spotify.audio_features(features_user)
    feature_df = pd.DataFrame(feats)
    scaled_input= scaler.transform(feature_df.select_dtypes(exclude = object))
    x = kmeans.predict(scaled_input)[0]
    feedback=all_features[all_features["cluster"] == x].sample(3)['id'].to_list()
    for song in feedback:
        name = spotify.track(song)
        list_feedback.append(name['name'])
    print(list_feedback[0])
    print(list_feedback[1])
    print(list_feedback[2])